In [ ]:
# Installation and Imports

!pip install -q ragas google-generativeai langchain-google-genai langchain-huggingface datasets evaluate pandas nbconvert

import os, sys
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Tuple
import nbformat
from nbconvert import PythonExporter
import pandas as pd
from datasets import load_dataset, Dataset
from tqdm.auto import tqdm
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [ ]:
# =============================================================================
# 1. Mount Google Drive and Define Paths
# =============================================================================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

DRIVE_ROOT = Path("/content/drive/MyDrive/Colab Notebooks")

# Notebook filenames
STEP6_NAIVE_NOTEBOOK = DRIVE_ROOT / "Naive_RAG_&_Evaluation.ipynb"
STEP6_ENHANCED_NOTEBOOK = DRIVE_ROOT / "Advanced_RAG.ipynb"

# Check they exist
if not STEP6_NAIVE_NOTEBOOK.exists():
    raise FileNotFoundError(f"Naive notebook not found: {STEP6_NAIVE_NOTEBOOK}")
if not STEP6_ENHANCED_NOTEBOOK.exists():
    raise FileNotFoundError(f"Enhanced notebook not found: {STEP6_ENHANCED_NOTEBOOK}")

print(f"Naive notebook:    {STEP6_NAIVE_NOTEBOOK}")
print(f"Enhanced notebook: {STEP6_ENHANCED_NOTEBOOK}")

# Create output directory
STEP6_OUTPUT_DIR = DRIVE_ROOT / "step6_outputs"
STEP6_OUTPUT_DIR.mkdir(exist_ok=True)

print(f"\nArtifacts will be saved to: {STEP6_OUTPUT_DIR}")

Mounted at /content/drive
Naive notebook:    /content/drive/MyDrive/Colab Notebooks/Naive_RAG_&_Evaluation.ipynb
Enhanced notebook: /content/drive/MyDrive/Colab Notebooks/Advanced_RAG.ipynb

Artifacts will be saved to: /content/drive/MyDrive/Colab Notebooks/step6_outputs


In [ ]:
# =============================================================================
# 2. Convert Notebooks to Python Scripts
# =============================================================================

def notebook_to_python(nb_path: Path, output_path: Path):
    """Convert a Jupyter notebook to a Python script."""
    print(f"\nConverting {nb_path.name}...")

    with open(nb_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    exporter = PythonExporter()
    python_code, _ = exporter.from_notebook_node(notebook)

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(python_code)

    print(f"  Saved to: {output_path}")
    return output_path

# Define output Python script paths
NAIVE_PY = STEP6_OUTPUT_DIR / "naive_system.py"
ENHANCED_PY = STEP6_OUTPUT_DIR / "enhanced_system.py"

# Convert notebooks
notebook_to_python(STEP6_NAIVE_NOTEBOOK, NAIVE_PY)
notebook_to_python(STEP6_ENHANCED_NOTEBOOK, ENHANCED_PY)


Converting Naive_RAG_&_Evaluation.ipynb...
  Saved to: /content/drive/MyDrive/Colab Notebooks/step6_outputs/naive_system.py

Converting Advanced_RAG.ipynb...
  Saved to: /content/drive/MyDrive/Colab Notebooks/step6_outputs/enhanced_system.py


PosixPath('/content/drive/MyDrive/Colab Notebooks/step6_outputs/enhanced_system.py')

In [ ]:
# =============================================================================
# 3. Import Converted Modules
# =============================================================================

def import_python_module(module_name: str, file_path: Path):
    """Import a Python module from file."""
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

print("\nImporting modules...")
naive_mod = import_python_module("naive_rag_system", NAIVE_PY)
enhanced_mod = import_python_module("enhanced_rag_system", ENHANCED_PY)
print("✓ Modules imported")


Importing modules...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 10.5 MB/s eta 0:00:00
Loading dataset from parquet...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



DATASET STATISTICS
Passage length statistics:
  Min: 1 characters
  Max: 2515 characters
  Mean: 389.8 characters
  Median: 299.0 characters

Null values per column:
passage    0
dtype: int64

Shape before dropping nulls: (3200, 1)
Shape after dropping nulls: (3200, 1)

Using 1000 passages for RAG system

Loading QA dataset...


README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

data/test.parquet/part.0.parquet:   0%|          | 0.00/54.4k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Reranker ready

────────────────────────────────────────────────────────────────────────────────
TESTING RAG SYSTEM - MULTIPLE QUERIES
────────────────────────────────────────────────────────────────────────────────

TEST QUERY #1
Query: Was Abraham Lincoln the sixteenth President of the United States?

Question: Was Abraham Lincoln the sixteenth President of the United States?


Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors



Query variations (2):
  1. Was Abraham Lincoln the sixteenth President of the United States?
  2. Abraham Lincoln was the sixteenth president of the United States.
Retrieved 35 unique candidates
Reranked to top 5

Context assembled: 1936 chars from 4 chunks

────────────────────────────────────────────────────────────────────────────────
RESULTS
────────────────────────────────────────────────────────────────────────────────

Answer: Abraham Lincoln (February 12, 1809 â April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination.

Source Citations:
┌──────┬─────────────┬──────────────┐
│ Rank │  Chunk ID   │   Relevance  │
├──────┼─────────────┼──────────────┤
│  1   │ 278-0       │      10.3089 │
│  2   │ 319-0       │       8.1220 │
│  3   │ 198-0       │      -0.4495 │
│  4   │ 383-0       │      -0.7348 │
└──────┴─────────────┴──────────────┘

────────────────────────────────────────────────────────────────────────────────



In [ ]:
# =============================================================================
# 4. Extract Components from Notebooks
# =============================================================================

print("\nExtracting components from notebooks...")

# From Naive RAG (Milvus-based)
milvus_client = getattr(naive_mod, 'milvus_client', None)
naive_chunks = getattr(naive_mod, 'document_chunks', None)
naive_embedder = getattr(naive_mod, 'embedding_model', None)
naive_pipeline = getattr(naive_mod, 'flan_pipeline', None)
naive_search_fn = getattr(naive_mod, 'search_similar_chunks', None)

# From Enhanced RAG (FAISS-based)
enhanced_chunks = getattr(enhanced_mod, 'all_chunks', None)
enhanced_encoder = getattr(enhanced_mod, 'encoder', None)
enhanced_pipeline = getattr(enhanced_mod, 'text_gen', None)
enhanced_rag_fn = getattr(enhanced_mod, 'generate_rag_answer', None)

# Get prompting strategy class
try:
    PromptStrategies = getattr(naive_mod, 'PromptingStrategies', None)
    persona_prompt = PromptStrategies.persona_prompt if PromptStrategies else None
except:
    persona_prompt = None

print(f"  Naive chunks: {len(naive_chunks) if naive_chunks else 0}")
print(f"  Enhanced chunks: {len(enhanced_chunks) if enhanced_chunks else 0}")
print(f"  Milvus client: {'✓' if milvus_client else '✗'}")
print(f"  Functions: {'✓' if all([naive_search_fn, enhanced_rag_fn, persona_prompt]) else '✗'}")


Extracting components from notebooks...
  Naive chunks: 1289
  Enhanced chunks: 1289
  Milvus client: ✓
  Functions: ✓


In [ ]:
# =============================================================================
# 5. Environment Setup and LLM Configuration
# =============================================================================

# Define API key
GEMINI_API_KEY = "Replace with your own key"
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.0,
    google_api_key=GEMINI_API_KEY
)

print("✓ Gemini configured with gemini-2.5-pro")

# Load test questions
qa_dataset = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")
test_questions = qa_dataset["test"]

NUM_EVAL_SAMPLES = 100
eval_questions = test_questions.select(range(NUM_EVAL_SAMPLES))

print(f"✓ Loaded {len(eval_questions)} evaluation questions")

✓ Gemini configured with gemini-2.5-pro
✓ Loaded 100 evaluation questions


In [ ]:
# =============================================================================
# 6. Define RAG Response Functions
# =============================================================================

def naive_rag_response(question: str) -> Tuple[str, List[str]]:
    """Execute naive RAG using Milvus (top-1, persona prompt)."""
    try:
        results = naive_search_fn(question, num_results=1)
        if not results:
            return "I don't know.", [""]

        context = results[0][1]
        prompt = persona_prompt(context, question)
        output = naive_pipeline(prompt, max_new_tokens=128, temperature=0.2)
        answer = output[0]["generated_text"].strip()

        return answer, [context]
    except Exception as e:
        print(f"Naive error: {e}")
        return "Error", [""]

def enhanced_rag_response(question: str) -> Tuple[str, List[str]]:
    """Execute enhanced RAG (query rewriting + reranking)."""
    try:
        answer, refs = enhanced_rag_fn(question)
        contexts = []
        for ref in refs[:5]:
            if isinstance(ref, dict):
                chunk_id = ref.get('id', '')
                for chunk in enhanced_chunks:
                    if chunk['id'] == chunk_id:
                        contexts.append(chunk['text'])
                        break
        return answer, contexts if contexts else [""]
    except Exception as e:
        print(f"Enhanced error: {e}")
        return "Error", [""]

print("✓ Response functions ready")

✓ Response functions ready


In [ ]:
# =============================================================================
# 7. Collect Evaluation Data
# =============================================================================

def process_question(idx: int, system_type: str) -> Dict:
    """Process one question."""
    try:
        question = eval_questions[idx]["question"]
        answer_data = eval_questions[idx].get("answer") or eval_questions[idx].get("answers")
        ground_truth = answer_data if isinstance(answer_data, str) else answer_data[0]

        if system_type == "naive":
            answer, contexts = naive_rag_response(question)
        else:
            answer, contexts = enhanced_rag_response(question)

        return {
            "question": question,
            "answer": answer,
            "contexts": contexts,
            "ground_truth": ground_truth
        }
    except Exception as e:
        print(f"Error Q{idx}: {e}")
        return None

def collect_data(system_type: str, num_workers: int = 10) -> List[Dict]:
    """Collect with multithreading."""
    print(f"\nCollecting {system_type.upper()} data...")

    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(process_question, i, system_type): i
                  for i in range(len(eval_questions))}

        for future in tqdm(as_completed(futures), total=len(futures), desc=system_type):
            result = future.result()
            if result:
                results.append(result)

    print(f"✓ Collected {len(results)} responses")
    return results

naive_data = collect_data("naive", num_workers=10)
enhanced_data = collect_data("enhanced", num_workers=10)

naive:   0%|          | 0/100 [00:00<?, ?it/s]

✓ Collected 100 responses


Question: Was Abraham Lincoln the sixteenth President of the United States?

Question: Did his mother die of pneumonia?

Question: Did Lincoln sign the National Banking Act of 1863?

Question: When did Lincoln begin his political career?

Question: How many long was Lincoln's formal education?

Question: What did The Legal Tender Act of 1862 establish?

Question: Who suggested Lincoln grow a beard?

Question: When did the Gettysburg address argue that America was born?

Question: Did Lincoln beat John C. Breckinridge in the 1860 election?

Question: Was Abraham Lincoln the first President of the United States?


enhanced:   0%|          | 0/100 [00:00<?, ?it/s]


Query variations (1):
  1. Did Lincoln sign the National Banking Act of 1863?
Retrieved 30 unique candidates
Reranked to top 5

Context assembled: 2009 chars from 4 chunks

Query variations (1):
  1. When did the Gettysburg address argue that America was born?

Query variations (1):
  1. How many long was Lincoln's formal education?
Retrieved 30 unique candidates

Query variations (1):
  1. When did Lincoln begin his political career?
Retrieved 30 unique candidates
Reranked to top 5

Context assembled: 2009 chars from 4 chunks
Retrieved 30 unique candidates
Reranked to top 5

Context assembled: 2009 chars from 4 chunks
Reranked to top 5

Context assembled: 1922 chars from 4 chunks

Query variations (2):
  1. What did The Legal Tender Act of 1862 establish?
  2. legal tendering

Question: Did Lincoln start his political career in 1832?
Retrieved 50 unique candidates
Reranked to top 5

Context assembled: 1913 chars from 5 chunks

Query variations (2):
  1. Did his mother die of pneumoni

In [ ]:
# ============================================================================
# Step 8: RAGAs Evaluation
# ============================================================================

def to_ragas(data: List[Dict]) -> Dataset:
    return Dataset.from_dict({
        "question": [d["question"] for d in data],
        "answer": [d["answer"] for d in data],
        "contexts": [d["contexts"] for d in data],
        "ground_truth": [d["ground_truth"] for d in data]
    })

naive_ds = to_ragas(naive_data)
enhanced_ds = to_ragas(enhanced_data)

# Configure embeddings for RAGAs
ragas_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("\nRunning RAGAs evaluation...")
metrics = [faithfulness, answer_relevancy, context_precision, context_recall]

print("Evaluating naive system...")
naive_scores = evaluate(
    naive_ds,
    metrics=metrics,
    llm=gemini_llm,
    embeddings=ragas_embeddings

print("Evaluating enhanced system...")
enhanced_scores = evaluate(
    enhanced_ds,
    metrics=metrics,
    llm=gemini_llm,
    embeddings=ragas_embeddings
)

print("✓ Evaluation complete")


Running RAGAs evaluation...
Evaluating naive system...


Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

Evaluating enhanced system...


Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

✓ Evaluation complete


In [ ]:
# =============================================================================
# 9. Save Raw RAGAs Scores
# =============================================================================

df = pd.DataFrame({
    'System': ['Naive', 'Enhanced'],
    'Faithfulness': [naive_scores['faithfulness'], enhanced_scores['faithfulness']],
    'Answer Relevancy': [naive_scores['answer_relevancy'], enhanced_scores['answer_relevancy']],
    'Context Precision': [naive_scores['context_precision'], enhanced_scores['context_precision']],
    'Context Recall': [naive_scores['context_recall'], enhanced_scores['context_recall']]
})

print("\n" + df.to_string(index=False))

df.to_csv(STEP6_OUTPUT_DIR / 'ragas_results_raw.csv', index=False)
print(f"\n✓ Saved to {STEP6_OUTPUT_DIR / 'ragas_results_raw.csv'}")


  System                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Faithfulness                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [ ]:
# =============================================================================
# 10. Aggregate RAGAs Metrics
# =============================================================================

# Calculate mean scores from the arrays
df = pd.DataFrame({
    'System': ['Naive', 'Enhanced'],
    'Faithfulness': [
        np.nanmean(naive_scores['faithfulness']),  # nanmean ignores NaN values
        np.nanmean(enhanced_scores['faithfulness'])
    ],
    'Answer Relevancy': [
        np.nanmean(naive_scores['answer_relevancy']),
        np.nanmean(enhanced_scores['answer_relevancy'])
    ],
    'Context Precision': [
        np.nanmean(naive_scores['context_precision']),
        np.nanmean(enhanced_scores['context_precision'])
    ],
    'Context Recall': [
        np.nanmean(naive_scores['context_recall']),
        np.nanmean(enhanced_scores['context_recall'])
    ]
})

print("\n" + "="*80)
print("RAGAS EVALUATION RESULTS (AGGREGATED)")
print("="*80)
print("\n" + df.to_string(index=False))

df.to_csv(STEP6_OUTPUT_DIR / 'ragas_results.csv', index=False)
print(f"\n✓ Saved to {STEP6_OUTPUT_DIR / 'ragas_results.csv'}")


RAGAS EVALUATION RESULTS (AGGREGATED)

  System  Faithfulness  Answer Relevancy  Context Precision  Context Recall
   Naive      0.427083          0.189883           0.380000            0.36
Enhanced      0.670000          0.270800           0.602458            0.63

✓ Saved to /content/drive/MyDrive/Colab Notebooks/step6_outputs/ragas_results.csv
